### old russian dictionary 

In [1]:
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
import pickle
import requests
import re

pd.options.display.max_rows = 1000

In [2]:
def get_soup(url, simple_encoding=False):
    req = requests.get(url)
    if not simple_encoding:
        req.encoding = 'utf-8'
    soup = BeautifulSoup(req.text, 'html.parser')
    return soup

In [47]:
i = 0
url = 'https://rus-old-russian-dict.slovaronline.com/2-%D0%B0%D0%B1%D0%B8%D1%A5'
df = list()

with open(f"dict_11_14.csv", "w", encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file, delimiter='\t')    
    while url:
        soup = get_soup(url)
        word = soup.find("strong").text
        full_text = soup.find("blockquote").text.strip().replace("\t", " ")
        writer.writerow((word, full_text, url))
        df.append({"word": word, "text": full_text, "url": url})
        i += 1
        url = soup.find("link", {"rel": "next"})['href']


In [61]:
pd.read_csv("dict_11_14_0.csv", names=["text", "next_url"], sep="\t")

,text,next_url
АБИѤ,"АБИѤ (857) нар. Тогда, тотчас; теперь: И тоу а...",https://rus-old-russian-dict.slovaronline.com/...
АБО,"АБО (1*) союз. В причин. знач. Так как, ибо: б...",https://rus-old-russian-dict.slovaronline.com/...
АБЫ,АБЫ (91) союз.1.В целевом знач. Чтобы: а кɤзек...,https://rus-old-russian-dict.slovaronline.com/...


In [67]:
# shortening described in the book
shorts = """вин. — винительный (падеж)
вм. — вместо
возвр. — возвратный
вопр. — вопросительный
воскл .— восклицательный
вр. — время 
выделит. — выделительный 
гадат.— гадательная (приписка) 
геогр. — географический 
гл. — глагол 
грен. — греческий 
дат. — дательный (падеж) 
дв. — двойственное число 
действ. — действительный залог 
доп. — дополнение 
дополнит. — дополнительный 
др. — другой 
ед. — единственное число 
жен. — женский род 
зд. — здесь 
зная.— значение 
изд.— издание 
изъявит.— изъявительное (наклонение)
изъяснит.— изъяснительный
им. — имя 
имен. — именительный (падеж)
инф. — инфинитив
колин. — количественный
конкр. — конкретный
кратк. — краткий
л. — лицо 
ласкат. — ласкательный
личн. — личный
межд. — междометие
мест. — местоимение
местн. — местный (падеж)
мн. — множественное число
муж. — мужской род
модальн. — модальный
накл.— наклонение
нар. — наречие
наст. — настоящее время 
начинат. — начинательный
неопр. — неопределенный
нескл. — несклоняемый
образн. — образное (употребление)
ок. — около 
определит. — определительный
отвл. — отвлеченный
отн. — относящийся 
относит. — относительный
отриц. — отрицательный
п. — падеж 
перен. — переносное (употребление) 
перечислит. — перечислительный 
побудит.— побудительный 
повелит.— повелительное (наклонение) 
подчинит.— подчинительное (наклонение) 
поздн. — поздний (список) 
полн. — полный 
пор. — порядковый 
пр. — прилагательное 
прав. — правильно 
предик. — предикатив 
предл. — предлог 
предлож. — предложение 
предположит. — предположительный 
придат.— придаточное (предложение) 
прилож. — приложение 
присоединит. — присоединительный 
притяж. — притяжательный 
прич. — причастие 
причин. — причинный 
прозе. — прозвище 
противит. — противительный 
противоп. — противоположное 
прош. — прошедшее время 
разделит. — разделительный 
род. — родительный (падеж) 
рук. — рукопись 
с. — существительное 
собир. — собирательный 
собств. — собственный 
соедин. — соединительный 
сопостав. — сопоставительный 
сослагат.— сослагательное (наклонение) 
сост. — состав 
соч. — сочетание 
сочин. — сочинительный 
ср. — сравни 
сравн. — сравнительный 
среди. — средний род 
степ. — степень 
страд. — страдательный залог 
твор. — творительный (падеж) 
указат. — указательный 
уменым. — уменьшительный 
уничижит.— уничижительный
употр.— употребление
усилит.— усилительны
уcл. — условный
уступ. — уступительный 
ч. — частица 
числ. — слово с числовым значением"""

shortenings = dict(tuple(y.strip() for y in x.split("—")) for x in shorts.split("\n"))

In [23]:
s = df.text.loc[0]

In [91]:
def parse_raw_text(s):
    
    result = dict()
    
    num = re.search(r"\(.+?\)", s)
    if not num:
        point = re.search(r"см").span()[0]
        if not point:
            print(s)
            raise AssertionError
        dict_word = s[:point].strip()
        result["dict_word"] = dict_word
        return result
    
    dict_word = s[:num.span()[0]].strip()
    quantity = int(re.search(r"\d+", num[0])[0])
    rest = s[num.span()[1]:].strip().split()
    for i, part in enumerate(rest):
        print(part)
        if part.startswith("-"):
            print("has -")
            result.setdefault("alt", list()).append(part)
        elif part in shortenings:
            print("short")
            result.setdefault("info", list()).append(part)
        elif part.endswith(":"):
            print("end")
            result.setdefault("definition", list()).append(part[:-1])
            result["definition"] = " ".join(result["definition"])
            if len(rest) > (i + 1):
                result["leftovers"] = " ".join(rest[(i + 1):])
            return result
        else:
            print("def")
            result.setdefault("definition", list()).append(part)

In [93]:
parse_raw_text(s)

нар.
short
Тогда,
def
тотчас;
def
теперь:
end


{'info': ['нар.'],
 'definition': 'Тогда, тотчас; теперь',
 'leftovers': 'И тоу абиѥ отъкрышѩ || ѥдинъ. отъ златыхъ онѣхъ ларевъ. И начѩша искладати. срачицѣ и свиты цс҃рьскы˫а. Изб 1076, 272—272 об.; и абиѥ вьсѣмъ весла ѡ(т) роукоу испадоша. и вьси отъ страха омьртвѣша. СкБГ XII, 13г; и абиѥ пакы дъждь прѣста и сл҃нце въси˫а. ЖФП XII, 64г; абиѥ же оученикомъ повелѣваѥть. свѣще при˫ати въ роукахъ и пѣ(с) исходьныхъ начѩти. (παραυτίκα) ЖФСтХII, 168; Гл҃ють ˫ако коѥмоужьдо чл҃вкоу ражающюсѩ. абиѥ бѣсъ соущьствьно съчетаѥтьсѩ ѡ(т) осоужени˫а адамова. (παραυτίκα) ΚΕ XII, 285б; по почѩтии же таковѣмь. абиѥ покади(т) по(п) окр(с)тъ одра одиною. УСт XII/ΧΙΙΙ, 274; единъ ѡ(т) воинъ. копьемь емоу ребра прободе. и абье изиде изъ ребра іс(с)ва кровь и в(о)да. КР 1284, 274в; Что оубо оч҃е аще придеть хотѩ взѩти ризоу мою. дамь ли ѥмоу абиѥ. (παραχρῆμα) ПНЧ 1296, 13 об.; ѥмоу же ѡтрокъ зълчью рыбьѥю ѡчи помаза. и абьѥ тоу прозрѣ. ПрЛ XIII, 82а; абиѥ ѡ(т)ложивъ сонъ ѡ(т) своѥю вѣкоу ѡ(т)вергъ стани 